In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

logging.basicConfig(format="%(asctime)s ; %(levelname)s ; %(message)s", level=logging.DEBUG)
logging.getLogger("scapy").setLevel(logging.CRITICAL)
logger = logging.getLogger("adAPT")

from typing import Tuple
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import L2
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from collections import Counter

2023-05-14 17:53:18.199489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 17:53:26,460 ; DEBUG ; Creating converter from 7 to 5
2023-05-14 17:53:26,461 ; DEBUG ; Creating converter from 5 to 7
2023-05-14 17:53:26,462 ; DEBUG ; Creating converter from 7 to 5
2023-05-14 17:53:26,463 ; DEBUG ; Creating converter from 5 to 7


In [3]:
b_pkl = "./data/benign_features.pkl"
m_pkl = "./data/malicious_features.pkl"

b_df = pd.read_pickle(b_pkl)
m_df = pd.read_pickle(m_pkl)

In [4]:
all_df = pd.concat([b_df, m_df])

In [5]:
all_df.sample(5)

,protocol,app_layer,source_addr,dest_addr,source_port,dest_port,proto_packet_length,proto_packet_cache,ip_packet_length,ip_packet_cache,...,base_domain_entropy,host_length,proto_packet_entropy,source_ip_class_a,source_ip_class_b,source_ip_class_c,dest_ip_class_a,dest_ip_class_b,dest_ip_class_c,malware
14395,IP,HTTPResponse,10.1.11.101,51.195.169.87,64823,8080,120,b'E\x00\x00x\xc81@\x00\x80\x06?\xce\n\x01\x0be...,120,b'E\x00\x00x\xc81@\x00\x80\x06?\xce\n\x01\x0be...,...,0.0,0,4.084184,10,10.1,10.1.11,51,51.195,51.195.169,1.0
59183,IP,Unknown,69.49.241.55,172.16.1.137,587,64195,40,"b'E\x00\x00(*\xaa\x00\x00\x80\x06,$E1\xf17\xac...",40,"b'E\x00\x00(*\xaa\x00\x00\x80\x06,$E1\xf17\xac...",...,0.0,0,3.821928,69,69.49,69.49.241,172,172.16,172.16.1,1.0
21426,IP,Unknown,162.246.19.18,172.16.1.137,465,64088,40,b'E\x00\x00(\xcf\xe2\x00\x00\x80\x06\x07L\xa2\...,40,b'E\x00\x00(\xcf\xe2\x00\x00\x80\x06\x07L\xa2\...,...,0.0,0,3.921928,162,162.246,162.246.19,172,172.16,172.16.1,1.0
4368,IP,HTTPResponse,104.21.50.34,10.9.20.144,80,49790,1280,b'E\x00\x05\x00\xdd\r\x00\x00\x80\x06\xa0\x1ah...,1280,b'E\x00\x05\x00\xdd\r\x00\x00\x80\x06\xa0\x1ah...,...,0.0,0,3.921928,104,104.21,104.21.50,10,10.9,10.9.20,1.0
5375,IP,HTTPResponse,10.1.11.101,51.195.169.87,64823,8080,1070,b'E\x00\x04.\xb73@\x00\x80\x06M\x16\n\x01\x0be...,1070,b'E\x00\x04.\xb73@\x00\x80\x06M\x16\n\x01\x0be...,...,0.0,0,4.121928,10,10.1,10.1.11,51,51.195,51.195.169,1.0


In [6]:
def prepare_df_for_ml(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    fields = [
        "protocol",
        "app_layer",
        "source_port",
        "dest_port",
        "proto_packet_length",
        "ip_packet_length",
        # "url",
        "base_domain",
        "tld",
        "url_entropy",
        "host_entropy",
        "base_domain_entropy",
        "host_length",
        "proto_packet_entropy",
        "source_ip_class_a",
        "source_ip_class_b",
        "source_ip_class_c",
        "dest_ip_class_a",
        "dest_ip_class_b",
        "dest_ip_class_c",
    ]
    new_df = df[fields]
    new_df.loc[:, ["source_port", "dest_port"]] = new_df[
        [
            "source_port",
            "dest_port",
        ]
    ].astype(str)
    new_df.loc[:, ["ip_packet_length", "source_port"]] = new_df[
        [
            "ip_packet_length",
            "source_port",
        ]
    ].astype(float)

    return pd.get_dummies(new_df)

In [7]:
y = all_df["malware"].values
input_values = all_df.drop("malware", axis=1)
prepped = prepare_df_for_ml(input_values)
X = np.asarray(prepped.values).astype("float64")

In [8]:
y.shape

(196981,)

In [9]:
X.shape

(196981, 6867)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.40, random_state=234)

In [11]:
X_test, X_cv, Y_test, Y_cv = train_test_split(X_test, Y_test, test_size=0.50, random_state=33)

In [12]:
print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"X_cv.shape: {X_cv.shape}")
print(f"Y_train.shape: {Y_train.shape}")
print(f"Y_test.shape: {Y_test.shape}")
print(f"Y_cv.shape: {Y_cv.shape}")

X_train.shape: (118188, 6867)
X_test.shape: (39396, 6867)
X_cv.shape: (39397, 6867)
Y_train.shape: (118188,)
Y_test.shape: (39396,)
Y_cv.shape: (39397,)


In [13]:
normalizer = tf.keras.layers.Normalization(axis=-1)

layer_1 = layers.Dense(units=9, input_shape=(X_train.shape[-1],), activation="relu", kernel_regularizer=L2(0.01))
layer_2 = layers.Dense(units=15, activation="relu", kernel_regularizer=L2(0.01))
layer_3 = layers.Dense(units=1, activation="sigmoid", kernel_regularizer=L2(0.01))

model = keras.Sequential([normalizer, layer_1, layer_2, layer_3])

In [14]:
model.compile(
    optimizer="adam",  # Optimizer
    # Loss function to minimize
    loss=losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.FalseNegatives()],
)

In [15]:
model.fit(X_train, Y_train, epochs=10)

Epoch 1/10
3694/3694 [==============================] - 11s 3ms/step - loss: 0.1896 - binary_accuracy: 0.9766 - false_negatives: 190.0000
Epoch 2/10
3694/3694 [==============================] - 9s 3ms/step - loss: 0.1501 - binary_accuracy: 0.9880 - false_negatives: 239.0000
Epoch 3/10
3694/3694 [==============================] - 11s 3ms/step - loss: 0.1455 - binary_accuracy: 0.9888 - false_negatives: 201.0000
Epoch 4/10
3694/3694 [==============================] - 11s 3ms/step - loss: 0.1372 - binary_accuracy: 0.9895 - false_negatives: 158.0000
Epoch 5/10
3694/3694 [==============================] - 10s 3ms/step - loss: 0.1296 - binary_accuracy: 0.9904 - false_negatives: 147.0000
Epoch 6/10
3694/3694 [==============================] - 10s 3ms/step - loss: 0.1279 - binary_accuracy: 0.9905 - false_negatives: 137.0000
Epoch 7/10
3694/3694 [==============================] - 10s 3ms/step - loss: 0.1279 - binary_accuracy: 0.9906 - false_negatives: 135.0000
Epoch 8/10
3694/3694 [=============

In [16]:
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test)
print("test loss, test acc:", results)

Evaluate on test data
1232/1232 [==============================] - 2s 2ms/step - loss: 0.1152 - binary_accuracy: 0.9937 - false_negatives: 25.0000
test loss, test acc: [0.11515635251998901, 0.9936795830726624, 25.0]


In [17]:
print("Evaluate on Cross Validation data")
results_cv = model.evaluate(X_cv, Y_cv)
print(f"cv loss and acc: {results_cv}")

Evaluate on Cross Validation data
1232/1232 [==============================] - 2s 2ms/step - loss: 0.1146 - binary_accuracy: 0.9939 - false_negatives: 22.0000
cv loss and acc: [0.11457856744527817, 0.9938573837280273, 22.0]


In [18]:
print("Evaluate on full data set")
results_full = model.evaluate(X, y)
print(f"cv loss and acc: {results_full}")

Evaluate on full data set
6156/6156 [==============================] - 12s 2ms/step - loss: 0.1152 - binary_accuracy: 0.9935 - false_negatives: 137.0000
cv loss and acc: [0.11520779132843018, 0.993496835231781, 137.0]
